In [3]:
import pandas as pd
import numpy as np

import platform
import matplotlib.pyplot as plt

%matplotlib inline

path = "c:/Windows/Fonts/malgun.ttf"
from matplotlib import font_manager, rc
if platform.system() == 'Darwin':
    rc('font', family='AppleGothic')
elif platform.system() == 'Windows':
    font_name = font_manager.FontProperties(fname=path).get_name()
    rc('font', family=font_name)
else:
    print('Unknown system... sorry~~~~')    

plt.rcParams['axes.unicode_minus'] = False

In [70]:
pip install webdriver-manager

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\bamxd\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [110]:
from selenium import webdriver
import time

In [116]:
driver = webdriver.Chrome('../driver/chromedriver')
driver.get("http://info.nec.go.kr/main/showDocument.xhtml?electionId=0000000000&topMenuId=VC&secondMenuId=VCCP09")

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\1146298774.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('../driver/chromedriver')


In [119]:
driver.switch_to.default_content()
driver.switch_to.frame('main')

NoSuchFrameException: Message: main


In [93]:
driver.find_element_by_id("electionType1").click()

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\68580914.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("electionType1").click()


In [94]:
driver.find_element_by_id("electionName").send_keys("제19대")

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\1885009769.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("electionName").send_keys("제19대")


In [95]:
driver.find_element_by_id("electionCode").send_keys("대통령선거")

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\1290631384.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_id("electionCode").send_keys("대통령선거")


In [96]:
sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")
sido_list = sido_list_raw.find_elements_by_tag_name("option")
sido_names_values = [option.text for option in sido_list]
sido_names_values = sido_names_values[2:]
sido_names_values

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\2909572938.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  sido_list_raw = driver.find_element_by_xpath("""//*[@id="cityCode"]""")


['서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [97]:
import re

def get_num(tmp):
    return float(re.split('\(', tmp)[0].replace(',',''))

In [106]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

wait = WebDriverWait(driver, 10)

def move_sido(name):
    element = driver.find_element_by_id("cityCode")
    element.send_keys(name)
    make_xpath = """//*[@id="searchBtn"]"""
    wait.until(EC.element_to_be_clickable((By.XPATH,make_xpath)))
    driver.find_element_by_xpath(make_xpath).click() 

In [107]:
def append_data(df, sido_name, data):
    for each in df[0].values[1:]:
        data['광역시도'].append(sido_name)
        data['시군'].append(each[0])
        data['pop'].append(get_num(each[2]))
        data['moon'].append(get_num(each[3]))
        data['hong'].append(get_num(each[4]))
        data['ahn'].append(get_num(each[5]))

In [108]:
election_result_raw = {'광역시도' : [],
                       '시군' : [],
                       'pop' : [],
                       'moon' : [],
                       'hong' : [],    
                       'ahn' : [] }

In [109]:
from bs4 import BeautifulSoup

for each_sido in sido_names_values:
    move_sido(each_sido)
    
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table = soup.find('table')
    
    df = pd.read_html(str(table))
    
    append_data(df, each_sido, election_result_raw)

C:\Users\bamxd\AppData\Local\Temp\ipykernel_45136\3943248875.py:8: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  element = driver.find_element_by_id("cityCode")


TimeoutException: Message: 


In [ ]:
election_result = pd.DataFrame(election_result_raw, 
                               columns=['광역시도', '시군', 'pop', 'moon','hong','ahn'])
election_result

In [32]:
election_result.to_csv('../data/05. election_result.csv', encoding='utf-8', sep=',')

In [33]:
driver.close()

In [35]:
election_result = pd.read_csv('../data/05. election_result.csv', encoding='utf-8', 
                              index_col=0)
election_result.head()

,광역시도,시군,pop,moon,hong,ahn


In [36]:
sido_candi =  election_result['광역시도']
sido_candi = [name[:2] if name[:2] 
                                          in ['서울','부산','대구','광주','인천','대전','울산']
                                    else '' for name in sido_candi]

In [37]:
def cut_char_sigu(name):
    return name if len(name)==2 else name[:-1]

In [38]:
import re

sigun_candi = ['']*len(election_result)

for n in election_result.index:
    each = election_result['시군'][n]
    if each[:2] in ['수원', '성남','안양','안산','고양',
                            '용인','청주','천안','전주','포항','창원']:
        sigun_candi[n] = re.split('시', each)[0]+' '+ \
                                                        cut_char_sigu(re.split('시', each)[1])
    else:
        sigun_candi[n] = cut_char_sigu(each)
        
sigun_candi

[]

In [39]:
ID_candi = [sido_candi[n]+' '+sigun_candi[n] for n in range(0,len(sigun_candi))]

ID_candi = [name[1:] if name[0]==' ' else name for name in ID_candi]
ID_candi = [name[:2] if name[:2]=='세종' else name for name in ID_candi]

ID_candi

[]

In [40]:
election_result['ID'] = ID_candi
election_result.head(10)

,광역시도,시군,pop,moon,hong,ahn,ID


In [41]:
election_result[['rate_moon','rate_hong','rate_ahn']] = \
            election_result[['moon','hong','ahn']].div(election_result['pop'],  axis=0)
election_result[['rate_moon','rate_hong','rate_ahn']] *= 100
election_result.head()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [42]:
election_result.sort_values(['rate_moon'], ascending=[False]).head(10)

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [43]:
election_result.sort_values(['rate_hong'], ascending=[False]).head(10)

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [44]:
election_result.sort_values(['rate_ahn'], ascending=[False]).head(10)

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [45]:
draw_korea = pd.read_csv('../data/05. draw_korea.csv', encoding='utf-8', 
                                                                                                 index_col=0)
draw_korea.head()

FileNotFoundError: [Errno 2] No such file or directory: '../data/05. draw_korea.csv'

In [46]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

NameError: name 'draw_korea' is not defined

In [47]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

NameError: name 'draw_korea' is not defined

In [48]:
election_result[election_result['ID'] == '고성']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [49]:
election_result.loc[125, 'ID'] = '고성(강원)'
election_result.loc[233, 'ID'] = '고성(경남)'

election_result[election_result['시군'] == '고성군']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [50]:
election_result[election_result['광역시도'] == '경상남도']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [51]:
election_result.loc[228, 'ID'] = '창원 합포'
election_result.loc[229, 'ID'] = '창원 회원'

election_result[election_result['광역시도'] == '경상남도']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [52]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

NameError: name 'draw_korea' is not defined

In [53]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

NameError: name 'draw_korea' is not defined

In [54]:
election_result[election_result['시군'] == '부천시']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [55]:
election_result.tail()

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn
125,NaN,NaN,NaN,NaN,NaN,NaN,고성(강원),NaN,NaN,NaN
233,NaN,NaN,NaN,NaN,NaN,NaN,고성(경남),NaN,NaN,NaN
228,NaN,NaN,NaN,NaN,NaN,NaN,창원 합포,NaN,NaN,NaN
229,NaN,NaN,NaN,NaN,NaN,NaN,창원 회원,NaN,NaN,NaN


In [56]:
ahn_tmp = election_result.loc[85, 'ahn']/3
hong_tmp = election_result.loc[85, 'hong']/3
moon_tmp = election_result.loc[85, 'moon']/3
pop_tmp = election_result.loc[85, 'pop']/3

rate_moon_tmp = election_result.loc[85, 'rate_moon']
rate_hong_tmp = election_result.loc[85, 'rate_hong']
rate_ahn_tmp = election_result.loc[85, 'rate_ahn']

election_result.loc[250] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 소사', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[251] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 오정', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]
election_result.loc[252] = [ahn_tmp, hong_tmp, moon_tmp, pop_tmp, 
                           '경기도', '부천시', '부천 원미', 
                           rate_moon_tmp, rate_hong_tmp, rate_ahn_tmp]

KeyError: 85

In [57]:
election_result[election_result['시군'] == '부천시']

,광역시도,시군,pop,moon,hong,ahn,ID,rate_moon,rate_hong,rate_ahn


In [58]:
election_result.drop([85], inplace=True)
election_result[election_result['시군'] == '부천시']

KeyError: '[85] not found in axis'

In [59]:
set(draw_korea['ID'].unique()) - set(election_result['ID'].unique())

NameError: name 'draw_korea' is not defined

In [60]:
set(election_result['ID'].unique()) - set(draw_korea['ID'].unique())

NameError: name 'draw_korea' is not defined

In [61]:
final_elect_data = pd.merge(election_result, draw_korea, how='left', on=['ID'])
final_elect_data.head()

NameError: name 'draw_korea' is not defined

In [62]:
final_elect_data['moon_vs_hong'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_hong']
final_elect_data['moon_vs_ahn'] = final_elect_data['rate_moon'] - \
                                                                final_elect_data['rate_ahn']
final_elect_data['ahn_vs_hong'] = final_elect_data['rate_ahn'] - \
                                                                final_elect_data['rate_hong']
final_elect_data.head()

NameError: name 'final_elect_data' is not defined

In [63]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[False]).head(10)

NameError: name 'final_elect_data' is not defined

In [64]:
final_elect_data.sort_values(['moon_vs_hong'], ascending=[True]).head(10)

NameError: name 'final_elect_data' is not defined